In [1]:
import numpy as np
import pandas as pd 
import nltk

In [2]:
df = pd.read_excel("ArticlesApril2017.xlsx")
df.head()

,abstract,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,NaN,58def1347c459f24986d7c80,716,By STEPHEN HILTNER and SUSAN LEHMAN,article,Finding an Expansive View of a Forgotten Peop...,"['Photography', 'New York Times', 'Niger', 'Fe...",3,Insider,2,2017-04-01 00:15:41,Unknown,One of the largest photo displays in Times his...,The New York Times,News,https://www.nytimes.com/2017/03/31/insider/nig...
1,NaN,58def3237c459f24986d7c84,823,By GAIL COLLINS,article,"And Now, the Dreaded Trump Curse","['United States Politics and Government', 'Tru...",3,OpEd,23,2017-04-01 00:23:58,Unknown,Meet the gang from under the bus.,The New York Times,Op-Ed,https://www.nytimes.com/2017/03/31/opinion/and...
2,NaN,58def9f57c459f24986d7c90,575,By THE EDITORIAL BOARD,article,Venezuela’s Descent Into Dictatorship,"['Venezuela', 'Politics and Government', 'Madu...",3,Editorial,22,2017-04-01 00:53:06,Unknown,A court ruling annulling the legislature’s aut...,The New York Times,Editorial,https://www.nytimes.com/2017/03/31/opinion/ven...
3,NaN,58defd317c459f24986d7c95,1374,By MICHAEL POWELL,article,Stain Permeates Basketball Blue Blood,"['Basketball (College)', 'University of North ...",3,Sports,1,2017-04-01 01:06:52,College Basketball,"For two decades, until 2013, North Carolina en...",The New York Times,News,https://www.nytimes.com/2017/03/31/sports/ncaa...
4,NaN,58df09b77c459f24986d7ca7,708,By DEB AMLEN,article,Taking Things for Granted,['Crossword Puzzles'],3,Games,0,2017-04-01 02:00:14,Unknown,In which Howard Barkin and Will Shortz teach u...,The New York Times,News,https://www.nytimes.com/2017/03/31/crosswords/...


In [14]:
import string
def clean_text(text):
    text = text.lower()
    result = ""
    for i in text:
        if i not in string.punctuation:
            result+=i
    return result

In [19]:
data = df['headline'].tolist()
df['cleaned'] = df['headline'].apply(clean_text)
data = df['cleaned'].tolist()
data

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuela’s descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted',
 'the caged beast awakens',
 'an everunfolding story',
 'o’reilly thrives as settlements add up',
 'mouse infestation',
 'divide in gop now threatens trump tax plan',
 'variety puzzle acrostic',
 'they can hit a ball 400 feet but play catch that’s tricky',
 'in trump country shock at trump budget cuts',
 'why is this hate different from all other hate',
 'pick your favorite ethical offender',
 'my son’s growing black pride',
 'jerks and the startups they ruin',
 'trump  needs  a brain',
 'manhood in the age of trump',
 'the value of a black college',
 'initial description',
 'rough estimates',
 'el pasatiempo nacional',
 'cooling off on a hot day at yankee stadium',
 'trump’s staff mixed politics and paydays',
 'a virtuoso rebuilding act requires everyone in tune',
 '‘homeland

In [39]:
import tensorflow as tf
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(data)
total_words= len(tokenizer.word_index)+1

In [40]:
input_sequence = []
for line in data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_seq = token_list[:i+1]
        input_sequence.append(n_gram_seq)
print(input_sequence[:10])
print(total_words)

[[169, 21], [169, 21, 653], [169, 21, 653, 359], [169, 21, 653, 359, 4], [169, 21, 653, 359, 4, 2], [169, 21, 653, 359, 4, 2, 654], [169, 21, 653, 359, 4, 2, 654, 170], [169, 21, 653, 359, 4, 2, 654, 170, 5], [169, 21, 653, 359, 4, 2, 654, 170, 5, 655], [6, 80]]
2484


In [41]:
max_seq_length = max([len(seq) for seq in input_sequence])
input_sequence = tf.keras.preprocessing.sequence.pad_sequences(input_sequence,maxlen=max_seq_length,padding='pre')
predictors,labels = input_sequence[:,:-1],input_sequence[:,-1]
label = tf.keras.utils.to_categorical(labels,total_words)
print(label)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [43]:
input_len = max_seq_length - 1
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(total_words,10,input_length=input_len),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(total_words,activation='softmax')
])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 10)            24840     
                                                                 
 lstm_1 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense (Dense)               (None, 2484)              250884    
                                                                 
Total params: 320124 (1.22 MB)
Trainable params: 320124 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
model.fit(
    predictors,
    label,
    epochs = 100,
    batch_size = 16,
    verbose = 1
)

Epoch 1/100
301/301 [==============================] - 9s 20ms/step - loss: 7.3866 - accuracy: 0.0299
Epoch 2/100
301/301 [==============================] - 6s 19ms/step - loss: 6.9085 - accuracy: 0.0314
Epoch 3/100
301/301 [==============================] - 7s 23ms/step - loss: 6.7661 - accuracy: 0.0337
Epoch 4/100
301/301 [==============================] - 5s 18ms/step - loss: 6.6407 - accuracy: 0.0426
Epoch 5/100
301/301 [==============================] - 7s 23ms/step - loss: 6.5078 - accuracy: 0.0453
Epoch 6/100
301/301 [==============================] - 7s 23ms/step - loss: 6.3603 - accuracy: 0.0509
Epoch 7/100
301/301 [==============================] - 7s 24ms/step - loss: 6.1958 - accuracy: 0.0513
Epoch 8/100
301/301 [==============================] - 6s 21ms/step - loss: 6.0413 - accuracy: 0.0544
Epoch 9/100
301/301 [==============================] - 6s 22ms/step - loss: 5.8874 - accuracy: 0.0609
Epoch 10/100
301/301 [==============================] - 8s 26ms/step - loss: 5.736

In [48]:
s = "The cat"
n = 5
for i in range(5):
    token_list = tokenizer.texts_to_sequences([s])[0]
    input_sequence = tf.keras.preprocessing.sequence.pad_sequences([token_list],maxlen = max_seq_length-1,padding = 'pre')
    predictx = model.predict(input_sequence)
    classx = np.argmax(predictx,axis = 1)
    for word,index in tokenizer.word_index.items():
        if index == classx:
            out = word
            break
    s += " " + out
print(s)

1/1 [==============================] - 0s 32ms/step
The cat cost of a speech not
